In [1]:
import requests
from bs4 import BeautifulSoup
import os
from astropy.io import fits
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from tqdm import tqdm
import time
from astropy.utils.data import get_pkg_data_filename
from astropy.convolution import convolve, Gaussian1DKernel

In [ ]:
base_url = "https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/sv3/dark/"

# Obtengo las carpetas y archivos FITS desde la página HTML
def get_folders_and_fits(url):
    response = requests.get(url)
    # Verifica si la respuesta falló y muestra un mensaje
    if response.status_code != 200:
        print("Verifica tu conexión a Internet para asegurarte de que esté funcionando correctamente.")
        return [], []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    folders = []
    fits_files = []

    # Busco todos los enlaces en la página
    for link in soup.find_all('a', href=True):
        href = link['href']
        text = link.text
        if href.endswith('/') and not text.startswith('..') and not href.endswith('logs/'):
            # Es una carpeta
            folder_name = text.rstrip('/')
            folders.append(folder_name)
        elif text.startswith('redrock-sv3-dark-') or text.startswith('coadd-sv3-dark-'):
            # Es un archivo FITS
            fits_files.append(text)
    
    return folders, fits_files

# Explorar los archivos de la ultima, tercera, capa. Aqui estan los .fits
def explore_third_layer_folders(url):
    _ , fits_files = get_folders_and_fits(url)
    nstype=[]
    # FITS coadd y redrock
    for fits_file in tqdm(fits_files):
        with open('urlspectra.txt', 'a') as archivo:
            if fits_file.startswith('redrock-sv3-dark-'): 
                red_rock_fits_urls = url + fits_file  #aqui obtengo todos los urls que me llevan a todos los fits de redrock
                archivo.write(red_rock_fits_urls+"\n")
            if fits_file.startswith('coadd-sv3-dark-'): 
                coadd_fits_urls = url + fits_file
                archivo.write(coadd_fits_urls+"\n")
        print("copiado")
            

    
# Explorar las carpetas de la segunda capa 10016/ 10145/ etc/
def explore_second_layer_folders(url):
    folders, _ = get_folders_and_fits(url)
    for folder in folders:
        folder_url = url + folder + '/'
        explore_third_layer_folders(folder_url)                
                
# Explora las carpetas de la primera capa: 100/ 101/ etc/
def explore_first_layer_folders(url):
    folders, _ = get_folders_and_fits(url)
    for folder in folders:
        folder_url = url + folder + '/'
        explore_second_layer_folders(folder_url)
        
        
# Comienzo la exploración desde la URL base en la primera capa
explore_first_layer_folders(base_url)

In [3]:
with open('urlspectra.txt', 'r') as archivo:
    stype,STYPE=[],[]
    z,Z=[],[]
    target, TARGET=[],[]
    Bflux, BFLUX=[],[]
    Rflux,RFLUX=[],[]
    Zflux,ZFLUX=[],[]

    lineas = archivo.readlines()  # Lee todas las líneas del archivo y las almacena en una lista
    for i in range(0,3, 2): #Este rango va de el primero al tercer fits es decir los primeros 20 fits con 10 readrock y 10
        # Imprime dos líneas a la vez (par de líneas)
        if i + 1 < len(lineas):
            coadd=fits.open(lineas[i])
            readrock=fits.open(lineas[i + 1])
            read_readrock=Table.read(readrock, hdu=1)
            read_coadd = Table.read(coadd, hdu=1) 
            targetID_coadd=read_coadd['TARGETID'].data #Target ID lista # Redshift lista
            spectype=read_readrock['SPECTYPE'].data    # Tipo espectral lista
            Z_r=read_readrock['Z'].data # 
            for j in range(len(targetID_coadd)): #elimina los targetID negativos
                if targetID_coadd[j]>=0:
                    stype.append(spectype[j])
                    z.append(Z_r[j])
                    target.append(targetID_coadd[j])
                    Bflux.append(coadd[4].data[j])
                    Rflux.append(coadd[9].data[j])
                    Zflux.append(coadd[14].data[j])
            #une los espectros de cada archivo fits
            TARGET+=target
            Z+=z
            STYPE+=stype
            BFLUX+= Bflux
            RFLUX+= Rflux
            ZFLUX+= Zflux
            
            
    # Define los tipos de datos
    dtype = [('SPECTYPE', 'S7'), ('Z', 'float'), ('TARGETID', 'float')]
    # Crea un arreglo estructurado de NumPy
    data = np.zeros(len(Z), dtype=dtype)
    data['SPECTYPE'] = STYPE
    data['Z'] = Z
    data['TARGETID'] = TARGET
    # Crea una tabla FITS usando fits.BinTableHDU
    hdu = fits.BinTableHDU.from_columns(data)
    # Crea los arreglos para 
    data_Bflux = np.array(BFLUX, dtype=np.float32)
    data_Rflux = np.array(RFLUX, dtype=np.float32)
    data_Zflux = np.array(ZFLUX, dtype=np.float32)
    # Crea un objeto ImageHDU
    hdub = fits.ImageHDU(data_Bflux)
    hdur = fits.ImageHDU(data_Rflux)
    hduz = fits.ImageHDU(data_Zflux)
    # Establece el nombre del HDU como 'B_FLUX'
    hdub.name = 'B_FLUX'
    hdur.name = 'R_FLUX'
    hduz.name = 'Z_FLUX'
    #Agrega al fits.
    nombre_archivo = 'DataDESI.fits'
    hdulist = fits.open(nombre_archivo, mode='append')
    hdulist.append(hdu)
    hdulist.append(hdub)
    hdulist.append(hdur)
    hdulist.append(hduz)
    print("Copiado")
    hdulist.close() 


Copiado


In [4]:
from astropy.table import Table
espc = fits.open('DataDESI.fits')
espc.info()

Filename: DataDESI.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     14   71R x 3C   [7A, D, D]   
  2  B_FLUX        1 ImageHDU         8   (2751, 71)   float32   
  3  R_FLUX        1 ImageHDU         8   (2326, 71)   float32   
  4  Z_FLUX        1 ImageHDU         8   (2881, 71)   float32   


In [5]:
print(Table.read(espc, hdu=1))

SPECTYPE           Z                   TARGETID       
-------- ---------------------- ----------------------
  GALAXY     1.0562666391262403 3.9633166985596424e+16
  GALAXY     0.5144821155151134 3.9633166985595224e+16
     QSO     1.5490694958672993  3.963316698559575e+16
     QSO     1.1257626518264834  3.963316698559469e+16
  GALAXY -0.0011324745399137303 3.9633166985595016e+16
  GALAXY      1.412257154321883 3.9633166985595704e+16
     QSO    0.15117289243645263  3.963316698559526e+16
  GALAXY     0.5890760823970248  3.963316698559476e+16
  GALAXY     0.5878826780448072 3.9633166985595464e+16
  GALAXY     1.3146877092416778  3.963316698559501e+16
     ...                    ...                    ...
     QSO     1.0408366243937803  3.963316698559511e+16
  GALAXY     1.3459637344366364  3.963316698559478e+16
  GALAXY     1.4163996443501743  3.963316698559559e+16
  GALAXY     1.1838060439667448  3.963316698559553e+16
  GALAXY    0.36609178441399526  3.963316698559486e+16
  GALAXY  

In [ ]:
print(Table.read(espc, hdu=1)["Spectype"].data)

In [ ]:
print(len(espc[2].data))
print(len(espc[4].data[0]))